# Solución Caso 1


## Conjuntos Necesarios


In [3]:
import pandas as pd
#Clientes y sus datos
clients_df = pd.read_csv('data/Clients.csv')
clients_dict = {f"C{int(row['ClientID'])}": [int(row['Product']),row['Longitude'],row['Latitude']] for _, row in clients_df.iterrows()}
print(clients_dict)

#Depositos y sus datos
deposits_df = pd.read_csv('data/Depots.csv')
# deposits_df = pd.read_csv('single_depot.csv')
deposits_dict = {f"D{int(row['DepotID'])}": [int(row['DepotID']),row['Longitude'],row['Latitude']] for _, row in deposits_df.iterrows()}
print(deposits_dict)

#Vehículos y sus datos
vehicles_df = pd.read_csv('data/Vehicles.csv')
type_mapping = {
    'gas car': 'GS',
    'ev': 'EV',
    'solar ev': 'EV',
    'drone': 'DR'
}
vehicles_dict = {}
for idx, row in enumerate(vehicles_df.itertuples(), start=1):
    vehicle_id = f"V{idx}"  # Claves como V1, V2, ...
    vehicles_dict[vehicle_id] = [type_mapping[row.VehicleType.lower()], row.Capacity, row.Range]
print(vehicles_dict)

#Datos de Costos por tipo de vehículo
vehicles_data_df = pd.read_csv('data/vehicles_data.csv')
vehicles_data_df.rename(
    columns={
        'Vehicle': 'Type',
        'Freight Rate [COP/km]': 'TF',
        'Time Rate [COP/min]': 'TT',
        'Daily Maintenance [COP/day]': 'CM',
        'Recharge/Fuel Cost [COP/(gal or kWh)]': 'CRC',
        'Recharge/Fuel Time [min/10 percent charge]': 'TR',
        'Avg. Speed [km/h]': 'VP',
        'Gas Efficiency [km/gal]': 'Gas_Efficiency',
        'Electricity Efficency [kWh/km]': 'Electricity_Efficiency'
    },
    inplace=True
)

vehicles_parameters = {}
for _, row in vehicles_data_df.iterrows():
    vehicle_type = type_mapping[row['Type'].lower()]
    vehicles_parameters[vehicle_type] = {
        'TF': row['TF'],
        'TT': row['TT'],
        'CM': row['CM'],
        'CRC': row['CRC'],
        'TR': row['TR'],
        'VP': row['VP'],
        'EC': row['Gas_Efficiency'] if not pd.isna(row['Gas_Efficiency']) else row['Electricity_Efficiency']
    }
print(vehicles_parameters)

#Matriz de distancias terrestres
distance_matrix_df = pd.read_csv('data/distance_matrix.csv', index_col=0)
# distance_matrix_df = pd.read_csv('distance_single_matrix.csv', index_col=0)
distance_dict = {
    (row, col): float(distance_matrix_df.loc[row, col])
    for row in distance_matrix_df.index
    for col in distance_matrix_df.columns
}
print({key: distance_dict[key] for key in list(distance_dict.keys())[:20]})

#Matriz de tiempos terrestres
duration_matrix_df = pd.read_csv('data/duration_matrix.csv', index_col=0)
# duration_matrix_df = pd.read_csv('duration_single_matrix.csv', index_col=0)
duration_dict = {
    (row, col): float(duration_matrix_df.loc[row, col])
    for row in duration_matrix_df.index
    for col in duration_matrix_df.columns
}
print({key: duration_dict[key] for key in list(duration_dict.keys())[:20]})

#Matriz de distancias aéreas
distance_air_df = pd.read_csv('data/distance_dron_matrix.csv', index_col=0)
# distance_air_df = pd.read_csv('distance_dron_single_matrix.csv', index_col=0)
distance_air_dict = {
    (row, col): float(distance_air_df.loc[row, col])
    for row in distance_air_df.index
    for col in distance_air_df.columns
}
print({key: distance_air_dict[key] for key in list(distance_air_dict.keys())[:20]})

{'C1': [13, -74.09893796560621, 4.59795431125545], 'C2': [15, -74.07557103763986, 4.687820646838871], 'C3': [12, -74.10708524062704, 4.70949446000624], 'C4': [15, -74.09727965657427, 4.605029068682624], 'C5': [20, -74.16464148202755, 4.648463876533332], 'C6': [17, -74.12083799988112, 4.662137416953968], 'C7': [17, -74.02213076607309, 4.697499030379109], 'C8': [20, -74.17207549744595, 4.649416884236942], 'C9': [20, -74.15615257246444, 4.606310650273935], 'C10': [15, -74.09041145358674, 4.557379705282216], 'C11': [17, -74.17802255204528, 4.591594072172954], 'C12': [12, -74.1015410917749, 4.7564172406324055], 'C13': [21, -74.09690889182339, 4.646217006050524], 'C14': [15, -74.1219200708342, 4.725912125314368], 'C15': [17, -74.0942948461378, 4.604168478560718], 'C16': [10, -74.11138839002187, 4.557320898243896], 'C17': [25, -74.12463941285208, 4.615869066082658], 'C18': [12, -74.12456164551857, 4.656402930181292], 'C19': [11, -74.04990580408855, 4.706188309535041], 'C20': [15, -74.12186680

## Modelo


In [4]:
from pyomo.environ import *

model = ConcreteModel()

# Conjuntos
unique_nodes = set(i for i, j in distance_dict.keys()).union(j for i, j in distance_dict.keys())
model.I = Set(initialize=unique_nodes)  # Nodos (orígenes y destinos)
model.PAIRS = Set(dimen=2, initialize=distance_dict.keys())  # Pares válidos (i, j)
model.K = Set(initialize=vehicles_dict.keys())  # Vehículos

# Parámetros
# Distancia y tiempo entre nodos
model.distance = Param(model.PAIRS, initialize=distance_dict)
model.duration = Param(model.PAIRS, initialize=duration_dict)
model.distance_air = Param(model.PAIRS, initialize=distance_air_dict)

# Parámetros del vehículo
model.TF = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TF'] for k in model.K})
model.TT = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TT'] for k in model.K})
model.CM = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CM'] for k in model.K})
model.CRC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CRC'] for k in model.K})
model.TR = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TR'] for k in model.K})
model.VP = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['VP'] for k in model.K})
model.EC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['EC'] for k in model.K})


# Variables de decisión
model.x = Var(model.I, model.I, model.K, domain=Binary)  # Si el vehículo k viaja de i a j
model.W = Var(model.K, domain=NonNegativeReals)  # Peso total transportado por vehículo k
model.u = Var(model.I, model.K, domain=NonNegativeReals)  # Posición secuencial en la ruta para evitar subtours


# Función objetivo
def total_cost_rule(model):
    # Costo por distancia (calculado desde los recorridos)
    C_total = sum(
        model.TF[k] * (
            model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]
        ) * model.x[i, j, k]
        for k in model.K for i in model.I for j in model.I if i != j
    )
    
    # Costo por tiempo
    C_tiempo = sum(
    model.TT[k] * (
        (model.distance[i, j] / 1000 / (model.VP[k] / 60)) if vehicles_dict[k][0] == 'DR' else (model.duration[i, j] / 60)
    ) * model.x[i, j, k]
    for k in model.K for i in model.I for j in model.I if i != j
)

    # Costo por peso transportado
    C_carga = sum(model.W[k] * 500 for k in model.K)
    
    # Costo de mantenimiento diario
    C_mantenimiento = sum(
    model.CM[k] * sum(
        model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j
    )
    for k in model.K
)

    # Costo total
    return C_total + C_tiempo + C_carga + C_mantenimiento


model.obj = Objective(rule=total_cost_rule, sense=minimize)

# Restricciones

# Salida desde un depósito
def salida_desde_deposito_rule(model, k):
    return sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j) <= 1
model.salida_desde_deposito = Constraint(model.K, rule=salida_desde_deposito_rule)

# Regreso al depósito de origen
def regreso_a_deposito_rule(model, k):
    return sum(model.x[j, d, k] for d in deposits_dict.keys() for j in model.I if d != j) == sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j)
model.regreso_a_deposito = Constraint(model.K, rule=regreso_a_deposito_rule)

# Regreso al mismo depósito de origen
def regreso_al_mismo_deposito_rule(model, k, d):
    if d in deposits_dict.keys():
        # Si el vehículo sale del depósito `d`, debe regresar a él
        return sum(model.x[d, j, k] for j in model.I if d != j) == sum(model.x[j, d, k] for j in model.I if d != j)
    else:
        return Constraint.Skip  # Ignorar para nodos que no son depósitos
# model.regreso_al_mismo_deposito = Constraint(model.K, deposits_dict.keys(), rule=regreso_al_mismo_deposito_rule)

# Flujo correcto entre nodos (clientes y depósitos)
def flujo_correcto_rule(model, i, k):
    if i in deposits_dict.keys():  # No aplica para depósitos
        return Constraint.Skip
    return sum(model.x[i, j, k] for j in model.I if i != j) == sum(model.x[j, i, k] for j in model.I if i != j)
model.flujo_correcto = Constraint(model.I, model.K, rule=flujo_correcto_rule)

# MTZ para evitar subtours
def mtz_rule(model, i, j, k):
    if i == j or i in deposits_dict.keys() or j in deposits_dict.keys():
        return Constraint.Skip
    n = len(model.I)
    return model.u[i, k] - model.u[j, k] + (n - 1) * model.x[i, j, k] <= n - 2
model.mtz = Constraint(model.I, model.I, model.K, rule=mtz_rule)

# Capacidad Máxima de Carga
def capacidad_carga_rule(model, k):
    return model.W[k] <= vehicles_dict[k][1]  # Capacidad máxima del vehículo k
model.capacidad_carga = Constraint(model.K, rule=capacidad_carga_rule)

# Demanda de los clientes
def demanda_clientes_rule(model, i):
    if i not in clients_dict.keys():
        return Constraint.Skip
    return sum(model.x[j, i, k] for k in model.K for j in model.I if j != i) == 1
model.demanda_clientes = Constraint(model.I, rule=demanda_clientes_rule)

# Capacidad y Demanda
def capacidad_demanda_rule(model, k):
    return model.W[k] == sum(clients_dict[i][0] * sum(model.x[i, j, k] for j in model.I if i != j) for i in clients_dict.keys())
model.capacidad_demanda = Constraint(model.K, rule=capacidad_demanda_rule)

# Distancia Máxima (Rango)
def distancia_maxima_rule(model, k):
    return sum(
        (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k]
        for i in model.I for j in model.I if i != j
    ) <= vehicles_dict[k][2]
model.distancia_maxima = Constraint(model.K, rule=distancia_maxima_rule)

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [ ]:
solver_name = "appsi_highs"
solver = SolverFactory(solver_name)
solver.options['parallel'] = 'on'
solver.options['threads'] = 6
solver.options['time_limit'] = 4000  # 1-hour time limit
solver.options['mip_rel_gap'] = 0.05  # 5% relative gap
result = solver.solve(model, tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-01, 4e+01]
  Cost   [5e+02, 3e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
6984 rows, 12542 cols, 77335 nonzeros  0s
6646 rows, 9110 cols, 58801 nonzeros  5s
6344 rows, 8576 cols, 55108 nonzeros  14s

Solving MIP model with:
   6344 rows
   8576 cols (8276 binary, 0 integer, 12 implied int., 288 continuous)
   55108 nonzeros
MIP-Timing:          17 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestS

## Resultados


### Archivo CSV Rutas


In [6]:
import csv

# Ruta del archivo CSV de resultados
csv_results_path = "results/reporte_rutas.csv"

# Crear el archivo CSV
with open(csv_results_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Escribir encabezados
    writer.writerow(["Vehículo", "Ruta", "Distancia Total (km)", "Carga Total (kg)"])

    # Generar rutas para cada vehículo
    for k in model.K:
        # Reconstruir la ruta ordenada
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si encontramos el nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break

        # Calcular distancia total y peso total transportado
        if len(ruta) > 1:  # Asegurarse de que hay al menos una ruta válida
            distancia_total = sum(
                (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
                for i, j in zip(ruta[:-1], ruta[1:])
            )
        else:
            distancia_total = 0.0

        carga_total = model.W[k].value if ruta else 0.0

        # Escribir la ruta al CSV
        writer.writerow([k, " -> ".join(ruta) if ruta else "Sin ruta", f"{distancia_total:.2f}", f"{carga_total:.2f}"])

print(f"Archivo CSV de rutas generado: {csv_results_path}")


Archivo CSV de rutas generado: results/reporte_rutas.csv


### Reporte Resultados


In [7]:
results_path = "results/reporte_resultados.txt"

with open(results_path, "w") as file:
    # Escribir el costo total
    file.write("=== Resultados ===\n")
    file.write(f"Costo total operativo: {model.obj():.2f} COP\n\n")

    # Escribir los recorridos por vehículo en orden
    file.write("Recorridos asignados por vehículo:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si se encuentra un nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break

        # Escribir la ruta ordenada
        if ruta:
            file.write(f"  Ruta: {' -> '.join(ruta)}\n")
        else:
            file.write("  No tiene asignaciones.\n")

    # Escribir distancias totales recorridas
    file.write("\nDistancias totales recorridas:\n")
    for k in model.K:
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si se encuentra un nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break
        if len(ruta) > 1:  # Asegurarse de que hay al menos una ruta válida
            distancia_total = sum(
                (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
                for i, j in zip(ruta[:-1], ruta[1:])
            )
        else:
            distancia_total = 0.0

        carga_total = model.W[k].value if ruta else 0.0
        file.write(f"  Vehículo {k}: {distancia_total:.2f} km\n")

    # Escribir pesos totales transportados
    file.write("\nPesos totales transportados:\n")
    for k in model.K:
        file.write(f"  Vehículo {k}: {model.W[k].value:.2f} kg\n")

print(f"Resultados escritos en el archivo: {results_path}")


Resultados escritos en el archivo: results/reporte_resultados.txt


### Mapa Interactivo


In [8]:
import folium
from folium.plugins import AntPath
from folium import FeatureGroup, LayerControl
import requests

# Crear un mapa centrado en una ubicación aproximada
centro_lat = deposits_df['Latitude'].mean()
centro_lon = deposits_df['Longitude'].mean()
mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

# Asignar colores únicos a los vehículos
colors = ['red', 'blue', 'green', 'purple', 'orange', 'black', 'darkred', 'darkblue']
vehicle_colors = {f"V{idx+1}": colors[idx % len(colors)] for idx in range(len(model.K))}

# Crear un grupo de características para cada vehículo
vehicle_groups = {k: FeatureGroup(name=f"Vehículo {k}") for k in model.K}

# Agregar marcadores para depósitos
for depot_id, (id, lon, lat) in deposits_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Depósito {depot_id}",
        icon=folium.Icon(color="darkblue", icon="warehouse", prefix="fa")
    ).add_to(mapa)

# Agregar marcadores para clientes
for client_id, (product, lon, lat) in clients_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Cliente {client_id} - Demanda {product}",
        icon=folium.Icon(color="green", icon="user", prefix="fa")
    ).add_to(mapa)

# Función para calcular ruta real con OSRM
def calcular_ruta_osrm(coord_inicio, coord_fin):
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_inicio[1]},{coord_inicio[0]};{coord_fin[1]},{coord_fin[0]}"
        params = {"overview": "full", "geometries": "geojson"}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            geometry = route["routes"][0]["geometry"]["coordinates"]
            return [(lat, lon) for lon, lat in geometry]
        else:
            print(f"Error OSRM: {response.status_code}")
            return [coord_inicio, coord_fin]
    except Exception as e:
        print(f"Error al calcular ruta OSRM: {e}")
        return [coord_inicio, coord_fin]

# Dibujar los recorridos de los vehículos con control de capas
for k in model.K:
    color = vehicle_colors[k]  # Asignar color único al vehículo
    grupo = vehicle_groups[k]

    for i in model.I:
        for j in model.I:
            if i != j and model.x[i, j, k].value > 0.5:  # Si hay un recorrido entre i y j
                # Obtener coordenadas de los nodos
                if i in clients_dict:
                    coord_i = (clients_dict[i][2], clients_dict[i][1])  # (lat, lon)
                else:
                    coord_i = (deposits_dict[i][2], deposits_dict[i][1])  # (lat, lon)
                if j in clients_dict:
                    coord_j = (clients_dict[j][2], clients_dict[j][1])  # (lat, lon)
                else:
                    coord_j = (deposits_dict[j][2], deposits_dict[j][1])  # (lat, lon)

                # Calcular la ruta (línea recta para drones, ruta OSRM para otros)
                ruta = [coord_i, coord_j] if vehicles_dict[k][0] == "DR" else calcular_ruta_osrm(coord_i, coord_j)

                # Dibujar la línea
                AntPath(
                    locations=ruta,
                    color=color,
                    weight=2.5,
                    opacity=1,
                    dash_array=[1, 20],
                    tooltip=(f"Vehículo {k}: {i} -> {j}<br>"
                             f"Distancia: {model.distance[i, j] / 1000:.2f} km<br>"
                             f"Peso transportado: {model.W[k].value:.2f} kg")
                ).add_to(grupo)

    # Agregar el grupo al mapa
    grupo.add_to(mapa)

# Agregar el control de capas
LayerControl(collapsed=False).add_to(mapa)


# Guardar el mapa en un archivo HTML
mapa.save("results/mapa_rutas.html")
print("Mapa guardado como 'mapa_rutas.html'")



Mapa guardado como 'mapa_rutas.html'


### PDF de Reporte


In [ ]:
!pip install fpdf
!pip install reportlab

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 35.8 MB/s eta 0:00:00


In [20]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, ListFlowable, ListItem
from reportlab.lib.units import cm
import re

def obtener_coordenadas(punto):
    if punto.startswith('D'):
        data = deposits_dict[punto]
        return (data[2], data[1])  # (lat, lon)
    elif punto.startswith('C'):
        data = clients_dict[punto]
        return (data[2], data[1])  # (lat, lon)
    return None

vehiculos_data = {}  # Guardar info general de rutas por vehiculo
distancias_data = {}
pesos_data = {}
costo_total = None
vehiculos_encontrados = []  # Para mantener el orden en que se encuentran los vehículos

with open('results/reporte_resultados.txt', 'r', encoding='utf-8') as f:
    lines = f.read().strip().split('\n')

current_vehicle = None
parse_distancias = False
parse_pesos = False

for line in lines:
    line = line.strip()
    # Costo total operativo
    if line.startswith("Costo total operativo:"):
        costo_match = re.search(r'Costo total operativo:\s+([\d.]+)', line)
        if costo_match:
            costo_total = float(costo_match.group(1))
    
    # Identificación de vehículos
    if line.startswith('Vehículo'):
        current_vehicle = re.search(r'(V\d+)', line)
        current_vehicle = current_vehicle.group(1) if current_vehicle else None
        if current_vehicle not in vehiculos_encontrados:
            vehiculos_encontrados.append(current_vehicle)
        # Inicializar vehículo sin ruta
        if current_vehicle not in vehiculos_data:
            vehiculos_data[current_vehicle] = {
                'ruta_completa': None,
                'segmentos': []
            }
    
    # Parseo de ruta
    if line.startswith('Ruta:'):
        ruta_puntos = [p.strip() for p in line.replace('Ruta:', '').split('->')]
        coords = [obtener_coordenadas(p) for p in ruta_puntos if p]
        partes_url = []
        for c in coords:
            lat, lon = c
            partes_url.append(f"{lat},{lon}")
        gmaps_url = "https://www.google.com/maps/dir/" + "/".join(partes_url)
        
        # Guardar ruta completa
        vehiculos_data[current_vehicle]['ruta_completa'] = (ruta_puntos, gmaps_url)
        
        # Crear enlaces punto a punto
        segmentos = []
        for i in range(len(ruta_puntos)-1):
            inicio = ruta_puntos[i]
            fin = ruta_puntos[i+1]
            c_inicio = obtener_coordenadas(inicio)
            c_fin = obtener_coordenadas(fin)
            seg_url = f"https://www.google.com/maps/dir/{c_inicio[0]},{c_inicio[1]}/{c_fin[0]},{c_fin[1]}"
            segmentos.append((inicio, fin, seg_url))
        vehiculos_data[current_vehicle]['segmentos'] = segmentos
    
    # Parseo de distancias
    if "Distancias totales recorridas:" in line:
        parse_distancias = True
        continue
    if parse_distancias:
        if line.startswith("Vehículo"):
            dist_match = re.search(r'Vehículo\s+(V\d+):\s+([\d.]+)\s+km', line)
            if dist_match:
                dist_veh = dist_match.group(1)
                dist_val = float(dist_match.group(2))
                distancias_data[dist_veh] = dist_val
        else:
            if not line or not line.startswith("Vehículo"):
                parse_distancias = False
    
    # Parseo de pesos
    if "Pesos totales transportados:" in line:
        parse_pesos = True
        continue
    if parse_pesos:
        if line.startswith("Vehículo"):
            peso_match = re.search(r'Vehículo\s+(V\d+):\s+([\d.-]+)\s+kg', line)
            if peso_match:
                peso_veh = peso_match.group(1)
                peso_val = float(peso_match.group(2))
                pesos_data[peso_veh] = peso_val
        else:
            if not line or not line.startswith("Vehículo"):
                parse_pesos = False

# Crear el PDF
doc = SimpleDocTemplate("results/informe.pdf", pagesize=A4)
styles = getSampleStyleSheet()
Story = []

# Agregar los logos
logo_uniandes = Image('data/LogoUnivLos_Andes02.png', width=4*cm, height=4*cm)
logo_otro = Image('data/logo_seneca.webp', width=4*cm, height=4*cm)
logos_table = Table([[logo_uniandes, logo_otro]], colWidths=[5*cm,5*cm])
logos_table.setStyle(TableStyle([
    ('ALIGN',(0,0),(-1,-1),'CENTER'),
    ('VALIGN',(0,0),(-1,-1),'MIDDLE')
]))
Story.append(logos_table)
Story.append(Spacer(1, 0.5*cm))

Story.append(Paragraph("Informe de Rutas", styles['Title']))
Story.append(Spacer(1, 0.5*cm))

# Sección Consultores
Story.append(Paragraph("Consultores:", styles['Heading2']))
Story.append(Spacer(1, 0.3*cm))
cons_list = ListFlowable([
    ListItem(Paragraph("Santiago Navarrete", styles['Normal'])),
    ListItem(Paragraph("Luis Ruiz", styles['Normal'])),
    ListItem(Paragraph("Andrea Lucia Galindo", styles['Normal']))
], bulletType='bullet', leftIndent=20)
Story.append(cons_list)
Story.append(Spacer(1, 0.5*cm))

# Costo total
if costo_total is not None:
    Story.append(Paragraph(f"Costo total operativo: {costo_total:.2f} COP", styles['Heading2']))
    Story.append(Spacer(1, 0.5*cm))

# Distancias totales
if distancias_data:
    Story.append(Paragraph("Distancias totales recorridas:", styles['Heading2']))
    Story.append(Spacer(1, 0.3*cm))
    tabla_dist = [["Vehículo", "Distancia (km)"]]
    for v, dist in distancias_data.items():
        tabla_dist.append([v, f"{dist:.2f} km"])
    t_dist = Table(tabla_dist, colWidths=[4*cm, 3*cm])
    t_dist.setStyle(TableStyle([
        ('BACKGROUND',(0,0),(-1,0), colors.gray),
        ('TEXTCOLOR',(0,0),(-1,0),colors.white),
        ('ALIGN',(0,0),(-1,-1),'LEFT'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('VALIGN',(0,0),(-1,-1),'TOP')
    ]))
    Story.append(t_dist)
    Story.append(Spacer(1, 0.5*cm))

# Pesos totales
if pesos_data:
    Story.append(Paragraph("Pesos totales transportados:", styles['Heading2']))
    Story.append(Spacer(1, 0.3*cm))
    tabla_pesos = [["Vehículo", "Peso (kg)"]]
    for v, peso in pesos_data.items():
        tabla_pesos.append([v, f"{peso:.2f} kg"])
    t_pesos = Table(tabla_pesos, colWidths=[4*cm, 3*cm])
    t_pesos.setStyle(TableStyle([
        ('BACKGROUND',(0,0),(-1,0), colors.gray),
        ('TEXTCOLOR',(0,0),(-1,0),colors.white),
        ('ALIGN',(0,0),(-1,-1),'LEFT'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('VALIGN',(0,0),(-1,-1),'TOP')
    ]))
    Story.append(t_pesos)
    Story.append(Spacer(1, 0.5*cm))

# Rutas asignadas por vehículo
Story.append(Paragraph("Rutas asignadas por vehículo:", styles['Heading2']))
Story.append(Spacer(1, 0.3*cm))

for veh in vehiculos_encontrados:
    Story.append(Paragraph(f"Vehículo {veh}:", styles['Heading3']))
    Story.append(Spacer(1, 0.3*cm))
    datos_veh = vehiculos_data.get(veh, None)
    if datos_veh and datos_veh['ruta_completa']:
        # Ruta punto a punto
        segmentos = datos_veh['segmentos']
        if segmentos:
            Story.append(Paragraph("Detalle de ruta punto a punto:", styles['Heading4']))
            Story.append(Spacer(1, 0.3*cm))
            tabla_segmentos = [["Desde", "Hasta", "Link"]]
            for (inicio, fin, link_seg) in segmentos:
                link_par = Paragraph(f'<link href="{link_seg}">{link_seg}</link>', styles['Normal'])
                tabla_segmentos.append([inicio, fin, link_par])
            
            t_seg = Table(tabla_segmentos, colWidths=[3*cm, 3*cm, 11*cm])
            t_seg.setStyle(TableStyle([
                ('BACKGROUND',(0,0),(-1,0), colors.gray),
                ('TEXTCOLOR',(0,0),(-1,0),colors.white),
                ('ALIGN',(0,0),(-1,-1),'LEFT'),
                ('GRID', (0,0), (-1,-1), 0.5, colors.black),
                ('VALIGN',(0,0),(-1,-1),'TOP')
            ]))
            Story.append(t_seg)
            Story.append(Spacer(1, 0.5*cm))
        
        # Ruta completa
        (ruta_puntos, gmaps_url) = datos_veh['ruta_completa']
        Story.append(Paragraph("Ruta completa:", styles['Heading4']))
        Story.append(Spacer(1, 0.3*cm))
        link_par = Paragraph(f'<link href="{gmaps_url}">{gmaps_url}</link>', styles['Normal'])
        Story.append(Paragraph(f"({ ' -> '.join(ruta_puntos) })", styles['Normal']))
        Story.append(Spacer(1, 0.2*cm))
        Story.append(link_par)
        Story.append(Spacer(1, 0.5*cm))
    else:
        # Sin rutas
        Story.append(Paragraph("No tiene rutas.", styles['Normal']))
        Story.append(Spacer(1, 0.5*cm))

doc.build(Story)
